# Introduction to CUDA programming


### Check the cuda compiler version

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples.git

In [ ]:
!cd cuda-samples/Samples/1_Utilities/deviceQuery && make


In [ ]:
!cd cuda-samples/Samples/1_Utilities/deviceQuery && ls
!cuda-samples/Samples/1_Utilities/deviceQuery/./deviceQuery

## nvcc for Jupyter notebook

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

# Kernel Questions

1. Given a vector performs 1D convolution.
2. Given a matrix, convert it to a vector and perform element-wise operations.
3. Given a vector, find the elementwise application of non-linear functions through approximations.
4. Given a vector, the first stage of stream compaction is performed. (using the boolean condition, convert the array to 0-1)
5. Perfo4rm matrix-vector multiplication 
6. Perform reduction in a matrix (Convert matrix to vector)
7. Using shared memory for matrix transpose
8. Matrix-Matrix Multiplication Using a Standard CUDA Kernel with Flattened Matrices
9. Use Thrust library to perform matrix-vector multiplication (self-study)

## Task 1: 1D Convolution on a Vector

In [ ]:
%%cuda
#include <stdio.h>

__global__ void convolve1D(float *input, float *output, float *kernel, int inputSize, int kernelSize) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int halfKernel = kernelSize / 2;
    if (idx >= inputSize) return;

    float sum = 0;
    for (int j = -halfKernel; j <= halfKernel; ++j) {
        int index = idx + j;
        if (index >= 0 && index < inputSize)
            sum += input[index] * kernel[halfKernel + j];
    }
    output[idx] = sum;
}

int main() {
    const int inputSize = 1024;
    const int kernelSize = 5;

    float *h_input = (float*)malloc(inputSize * sizeof(float));
    float *h_output = (float*)malloc(inputSize * sizeof(float));
    float h_kernel[kernelSize] = {0.2, 0.2, 0.2, 0.2, 0.2};

    float *d_input, *d_output, *d_kernel;
    cudaMalloc(&d_input, inputSize * sizeof(float));
    cudaMalloc(&d_output, inputSize * sizeof(float));
    cudaMalloc(&d_kernel, kernelSize * sizeof(float));

    cudaMemcpy(d_input, h_input, inputSize * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, h_kernel, kernelSize * sizeof(float), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (inputSize + blockSize - 1) / blockSize;
    convolve1D<<<numBlocks, blockSize>>>(d_input, d_output, d_kernel, inputSize, kernelSize);

    cudaMemcpy(h_output, d_output, inputSize * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_input); cudaFree(d_output); cudaFree(d_kernel);
    free(h_input); free(h_output);
    return 0;
}


## Task 2: Matrix to Vector Conversion and Element-wise Operations

In [ ]:
%%cuda
#include <stdio.h>

__global__ void matrixToVector(float *matrix, float *vector, int rows, int cols) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < rows * cols) {
        vector[idx] = matrix[idx] * 2.0; // Example element-wise operation
    }
}

int main() {
    const int rows = 1024;
    const int cols = 1024;
    const int size = rows * cols;

    float *h_matrix = (float*)malloc(size * sizeof(float));
    float *h_vector = (float*)malloc(size * sizeof(float));
    float *d_matrix, *d_vector;

    cudaMalloc(&d_matrix, size * sizeof(float));
    cudaMalloc(&d_vector, size * sizeof(float));

    cudaMemcpy(d_matrix, h_matrix, size * sizeof(float), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (size + blockSize - 1) / blockSize;
    matrixToVector<<<numBlocks, blockSize>>>(d_matrix, d_vector, rows, cols);

    cudaMemcpy(h_vector, d_vector, size * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_matrix); cudaFree(d_vector);
    free(h_matrix); free(h_vector);
    return 0;
}

## Task 3: Given a vector, find the elementwise application of non-linear functions through approximations.

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>
#include <math.h>

#define N 10

__device__ float approximate_tanh(float x) {
    // Approximate tanh(x) using Taylor expansion for small values of x
    return x - (x * x * x) / 3.0f + (2 * x * x * x * x * x) / 15.0f;
}

__global__ void apply_nonlinear(float *out, float *in, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        out[idx] = approximate_tanh(in[idx]);
    }
}

int main() {
    float in[N], out[N];
    float *d_in, *d_out;

    // Initialize input array with sample values
    for (int i = 0; i < N; i++) {
        in[i] = i * 0.1f;  // for example: 0.0, 0.1, 0.2, ..., 0.9
    }

    // Allocate device memory
    cudaMalloc((void**)&d_in, sizeof(float) * N);
    cudaMalloc((void**)&d_out, sizeof(float) * N);

    // Copy input to device
    cudaMemcpy(d_in, in, sizeof(float) * N, cudaMemcpyHostToDevice);

    // Launch kernel
    apply_nonlinear<<<1, N>>>(d_out, d_in, N);

    // Copy result back to host
    cudaMemcpy(out, d_out, sizeof(float) * N, cudaMemcpyDeviceToHost);

    // Print output
    printf("Elementwise tanh approximation:\n");
    for (int i = 0; i < N; i++) {
        printf("tanh(%f) ≈ %f\n", in[i], out[i]);
    }

    // Free device memory
    cudaFree(d_in);
    cudaFree(d_out);

    return 0;
}

## Task 4: Given a vector, the first stage of stream compaction is performed. (using the boolean condition, convert the array to 0-1)

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>

#define N 10

__global__ void stream_compaction(int *out, float *in, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        out[idx] = (in[idx] > 0.5f) ? 1 : 0;
    }
}

int main() {
    float in[N] = {0.1f, 0.4f, 0.6f, 0.7f, 0.2f, 0.9f, 0.3f, 0.5f, 0.8f, 0.15f};
    int out[N];
    float *d_in;
    int *d_out;

    // Allocate device memory
    cudaMalloc((void**)&d_in, sizeof(float) * N);
    cudaMalloc((void**)&d_out, sizeof(int) * N);

    // Copy input to device
    cudaMemcpy(d_in, in, sizeof(float) * N, cudaMemcpyHostToDevice);

    // Launch kernel
    stream_compaction<<<1, N>>>(d_out, d_in, N);

    // Copy result back to host
    cudaMemcpy(out, d_out, sizeof(int) * N, cudaMemcpyDeviceToHost);

    // Print output
    printf("Stream compaction (0-1 Boolean Conversion):\n");
    for (int i = 0; i < N; i++) {
        printf("Input: %f -> Output: %d\n", in[i], out[i]);
    }

    // Free device memory
    cudaFree(d_in);
    cudaFree(d_out);

    return 0;
}


## Task 5: Matrix-Vector Multiplication

In [ ]:
%%cuda
#include <stdio.h>

__global__ void matVecMult(float *matrix, float *vector, float *result, int rows, int cols) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < rows) {
        float sum = 0.0f;
        for (int col = 0; col < cols; ++col) {
            sum += matrix[row * cols + col] * vector[col];
        }
        result[row] = sum;
    }
}

int main() {
    const int rows = 1024;
    const int cols = 1024;

    float *h_matrix = (float*)malloc(rows * cols * sizeof(float));
    float *h_vector = (float*)malloc(cols * sizeof(float));
    float *h_result = (float*)malloc(rows * sizeof(float));

    float *d_matrix, *d_vector, *d_result;
    cudaMalloc(&d_matrix, rows * cols * sizeof(float));
    cudaMalloc(&d_vector, cols * sizeof(float));
    cudaMalloc(&d_result, rows * sizeof(float));

    cudaMemcpy(d_matrix, h_matrix, rows * cols * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_vector, h_vector, cols * sizeof(float), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (rows + blockSize - 1) / blockSize;
    matVecMult<<<numBlocks, blockSize>>>(d_matrix, d_vector, d_result, rows, cols);

    cudaMemcpy(h_result, d_result, rows * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_matrix); cudaFree(d_vector); cudaFree(d_result);
    free(h_matrix); free(h_vector); free(h_result);
    return 0;
}


##  Task 6: Perform reduction in a matrix (Convert matrix to vector)


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>

#define N 4  // Number of rows
#define M 4  // Number of columns

__global__ void reduce_matrix_to_vector(float *matrix, float *vector, int rows, int cols) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < rows) {
        float sum = 0.0;
        for (int col = 0; col < cols; col++) {
            sum += matrix[row * cols + col];
        }
        vector[row] = sum;  // Store the row sum in the vector
    }
}

int main() {
    float h_matrix[N][M] = {
        {1.0, 2.0, 3.0, 4.0},
        {5.0, 6.0, 7.0, 8.0},
        {9.0, 10.0, 11.0, 12.0},
        {13.0, 14.0, 15.0, 16.0}
    };
    float h_vector[N];  // Vector to store row sums
    float *d_matrix, *d_vector;

    // Allocate device memory
    cudaMalloc((void**)&d_matrix, sizeof(float) * N * M);
    cudaMalloc((void**)&d_vector, sizeof(float) * N);

    // Copy matrix to device
    cudaMemcpy(d_matrix, h_matrix, sizeof(float) * N * M, cudaMemcpyHostToDevice);

    // Launch kernel to reduce matrix to vector
    reduce_matrix_to_vector<<<1, N>>>(d_matrix, d_vector, N, M);

    // Copy result back to host
    cudaMemcpy(h_vector, d_vector, sizeof(float) * N, cudaMemcpyDeviceToHost);

    // Print output vector
    printf("Row sums (reduction result):\n");
    for (int i = 0; i < N; i++) {
        printf("Row %d sum: %f\n", i, h_vector[i]);
    }

    // Free device memory
    cudaFree(d_matrix);
    cudaFree(d_vector);

    return 0;
}


## Task 7: Using shared memory for matrix transpose

In [ ]:
%%cuda
#include <stdio.h>

#define TILE_WIDTH 32

__global__ void matrixTranspose(float *input, float *output, int width, int height) {
    __shared__ float tile[TILE_WIDTH][TILE_WIDTH];

    int x = blockIdx.x * TILE_WIDTH + threadIdx.x;
    int y = blockIdx.y * TILE_WIDTH + threadIdx.y;

    if (x < width && y < height) {
        int index_in = y * width + x;
        tile[threadIdx.y][threadIdx.x] = input[index_in];
    }

    __syncthreads();

    x = blockIdx.y * TILE_WIDTH + threadIdx.x;
    y = blockIdx.x * TILE_WIDTH + threadIdx.y;

    if (x < height && y < width) {
        int index_out = y * height + x;
        output[index_out] = tile[threadIdx.x][threadIdx.y];
    }
}

int main() {
    const int width = 1024;
    const int height = 1024;

    float *h_input = (float*)malloc(width * height * sizeof(float));
    float *h_output = (float*)malloc(width * height * sizeof(float));
    float *d_input, *d_output;

    cudaMalloc(&d_input, width * height * sizeof(float));
    cudaMalloc(&d_output, width * height * sizeof(float));

    cudaMemcpy(d_input, h_input, width * height * sizeof(float), cudaMemcpyHostToDevice);

    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
    dim3 dimGrid((width + TILE_WIDTH - 1) / TILE_WIDTH, (height + TILE_WIDTH - 1) / TILE_WIDTH);

    matrixTranspose<<<dimGrid, dimBlock>>>(d_input, d_output, width, height);

    cudaMemcpy(h_output, d_output, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_input); cudaFree(d_output);
    free(h_input); free(h_output);
    return 0;
}


## Task 8: Matrix-Matrix Multiplication Using a Standard CUDA Kernel with Flattened Matrices

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>

#define N 4  // Define matrix dimensions (N x N)

__global__ void matrixMultiplyKernel(float *A, float *B, float *C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < n && col < n) {
        float sum = 0;
        for (int i = 0; i < n; i++) {
            sum += A[row * n + i] * B[i * n + col];
        }
        C[row * n + col] = sum;
    }
}

int main() {
    float h_A[N * N] = {
        1, 2, 3, 4,
        5, 6, 7, 8,
        9, 10, 11, 12,
        13, 14, 15, 16
    };
    float h_B[N * N] = {
        16, 15, 14, 13,
        12, 11, 10, 9,
        8, 7, 6, 5,
        4, 3, 2, 1
    };
    float h_C[N * N];  // Output matrix

    float *d_A, *d_B, *d_C;

    // Allocate device memory
    cudaMalloc((void**)&d_A, sizeof(float) * N * N);
    cudaMalloc((void**)&d_B, sizeof(float) * N * N);
    cudaMalloc((void**)&d_C, sizeof(float) * N * N);

    // Copy matrices A and B to device
    cudaMemcpy(d_A, h_A, sizeof(float) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeof(float) * N * N, cudaMemcpyHostToDevice);

    // Define block and grid sizes
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Launch kernel for matrix-matrix multiplication
    matrixMultiplyKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Copy result back to host
    cudaMemcpy(h_C, d_C, sizeof(float) * N * N, cudaMemcpyDeviceToHost);

    // Print result
    printf("Resultant Matrix C:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%f ", h_C[i * N + j]);
        }
        printf("\n");
    }

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


## Task 9: Use Thrust library to perform matrix-vector multiplication (self-study)

In [ ]:
%%cuda
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/functional.h>
#include <iostream>

#define N 4  // Dimension of matrix and vector

struct matrix_vector_mult {
    float *matrix;
    int cols;

    matrix_vector_mult(float *matrix, int cols) : matrix(matrix), cols(cols) {}

    __host__ __device__ float operator()(int row_index, const float& vec_elem) {
        float sum = 0.0;
        for (int j = 0; j < cols; j++) {
            sum += matrix[row_index * cols + j] * vec_elem;
        }
        return sum;
    }
};

int main() {
    float h_matrix[N * N] = {
        1, 2, 3, 4,
        5, 6, 7, 8,
        9, 10, 11, 12,
        13, 14, 15, 16
    };
    float h_vector[N] = {1, 2, 3, 4};  // Vector for multiplication
    thrust::host_vector<float> h_result(N);

    thrust::device_vector<float> d_matrix(h_matrix, h_matrix + N * N);
    thrust::device_vector<float> d_vector(h_vector, h_vector + N);
    thrust::device_vector<float> d_result(N);

    // Perform matrix-vector multiplication using transform
    thrust::transform(thrust::counting_iterator<int>(0),
                      thrust::counting_iterator<int>(N),
                      d_result.begin(),
                      matrix_vector_mult(thrust::raw_pointer_cast(d_matrix.data()), N));

    // Copy result back to host and print
    thrust::copy(d_result.begin(), d_result.end(), h_result.begin());

    std::cout << "Resultant Vector:\n";
    for (int i = 0; i < N; i++) {
        std::cout << h_result[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}
